In [2]:
import os

In [3]:
%pwd

'd:\\Chicken_Disease_Classification\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Chicken_Disease_Classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [9]:
from src.utils import *
from src.constants import *

In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [15]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [14]:
pip install tensorflow


INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     --------------------- ------------------ 30.7/57.5 kB 1.4 MB/s eta 0:00:01
     --------------------------- ---------- 41.0/57.5 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 57.5/57.5 kB 507.9 kB/s eta 0:00:00
     ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/14.9 MB 1.7 MB/s eta 0:00:09
     ---------------------------------------- 0.1/14.9 MB 1.3 MB/s eta 0:00:12
     ---------------------------------------- 0.2/14.9 MB 1.2 MB/s eta 0:00:12
      --------------------------------------- 0.2/14.9 MB 1.3 MB/s eta 0:00:12
      --------------------------------------- 0.3/14.9 MB 1.4 MB/s eta 0:00:11
     - -------------------------------------- 0.4/14.9 MB 1.4 MB/s eta 0:

  You can safely remove it manually.


   -------------------------- ------------- 180.9/276.5 MB 1.9 MB/s eta 0:00:51
   -------------------------- ------------- 180.9/276.5 MB 1.9 MB/s eta 0:00:51
   -------------------------- ------------- 180.9/276.5 MB 1.9 MB/s eta 0:00:51
   -------------------------- ------------- 181.1/276.5 MB 1.8 MB/s eta 0:00:55
   -------------------------- ------------- 181.1/276.5 MB 1.8 MB/s eta 0:00:54
   -------------------------- ------------- 181.2/276.5 MB 1.8 MB/s eta 0:00:54
   -------------------------- ------------- 181.3/276.5 MB 1.7 MB/s eta 0:00:55
   -------------------------- ------------- 181.4/276.5 MB 1.7 MB/s eta 0:00:55
   -------------------------- ------------- 181.5/276.5 MB 1.7 MB/s eta 0:00:55
   -------------------------- ------------- 181.5/276.5 MB 1.7 MB/s eta 0:00:55
   -------------------------- ------------- 181.5/276.5 MB 1.7 MB/s eta 0:00:55
   -------------------------- ------------- 181.7/276.5 MB 1.7 MB/s eta 0:00:55
   -------------------------- ----------

In [16]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config


    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [19]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

d:\Chicken_Disease_Classification\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     